In [1]:
import os
import numpy as np
from tqdm import tqdm
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

import tensorflow as tf
import tensorflow.keras as tfk
from sklearn.metrics import *
import copy
import numpy as np
import matplotlib.pyplot as plt
import pickle

2023-04-30 12:45:29.891869: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None
if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()
    #strategy = tf.distribute.MirroredStrategy()
print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [3]:

num_aminoAcids = {0:'A', 1:'C', 2:'E', 3:'D', 4:'G', 5:'F', 6:'I', 7:'H', 8:'K', 9:'M', 10:'L',
            11:'N', 12:'Q', 13:'P', 14:'S', 15:'R', 16:'T', 17:'W', 18:'V', 19:'Y', 20:'X'}
embedding_keys = {
        '<pad>':0,
        'A':1,
        'C':2,
        'D':3,
        'E':4,
        'F':5,
        'G':6,
        'H':7,
        'I':8,
        'K':9,
        'L':10,
        'M':11,
        'N':12,
        'P':13,
        'Q':14,
        'R':15,
        'S':16,
        'T':17,
        'V':18,
        'W':19,
        'X':20,
        'Y':21,
        'B':22,
        'U':23,
        'Z':24,
        'O':25,
        '<mask>':26}
embedding_list = list(embedding_keys)

In [4]:
class NonZeroCCE(tf.keras.losses.Loss):
    def __init__(self,Weights=[],smoothing = None):
        super().__init__()
        if smoothing is not None:
            self.cce = tf.keras.losses.CategoricalCrossentropy(reduction='none', label_smoothing=smoothing)
        else:
            self.cce = tf.keras.losses.CategoricalCrossentropy(reduction='none')
        self.weights = np.array(Weights)
    def call(self, y_true, y_pred):
        if len(self.weights)>0:
            loss = self.cce(self.weights*y_true,y_pred)
        else:
            loss = self.cce(y_true,y_pred)
        return tf.math.reduce_sum(loss)/((y_true.shape[0]*y_true.shape[1])-float(tf.math.reduce_sum(tf.cast(tf.equal(tf.math.reduce_sum(y_true,-1),0),tf.int32))))

def Accuracy(y_true,y_pred):
    y_true = y_true.numpy()
    y_pred = y_pred.numpy()

    m = np.sum(y_true, axis=-1)
    y_t = np.argmax(y_true[m==1],axis=-1)
    y_p = np.argmax(y_pred[m==1],axis=-1)
    return accuracy_score(y_t,y_p)


In [5]:
tmp = np.load('./Data/Disordered_Regions_Train_Dataset.npz')
X = tmp['sequences']
X = np.argmax(X,axis=-1)
X_p = tmp['pssms']
Y = tmp['regions']
X_m = X!=0
X_m = X_m.astype(np.int32)
X_i = np.repeat(np.arange(1,X.shape[1]+1)[None,:], X.shape[0], axis=0)
X_i[X==0] = 0
max_len = X.shape[1]

train_ids = []
with open('./Data/DM3000_id.txt') as f:
    for l in f.readlines():
        if l[:3] == 'Dis':
            k = l.split('|')[1].split()[0]
        else:
            k = l.split()[0]
        train_ids += [k]
val_ids = []
with open('./Data/DM1229_id.txt') as f:
    for l in f.readlines():
        if l[:3] == 'Dis':
            k = l.split('|')[1].split()[0]
        else:
            k = l.split()[0]
        val_ids += [k]
a = tmp['seq_ids']
tr_ids_m = [True if i in train_ids else False for i in a]
val_ids_m = [True if i in val_ids else False for i in a]
len(val_ids),len(train_ids)

X1_val = X[val_ids_m][:,1:1863]
X4_val = X_m[val_ids_m][:,1:1863]
X2_val = X_p[val_ids_m][:,1:1863]
X3_val = X_i[val_ids_m][:,1:1863]
Y_val = Y[val_ids_m][:,1:1863]
X4_val = np.expand_dims(X4_val,-1)
X4_val.shape
#X1_val[X1_val==23]=0


X1 = X[tr_ids_m][:,1:1863]
X4 = X_m[tr_ids_m][:,1:1863]
X2 = X_p[tr_ids_m][:,1:1863]
X3 = X_i[tr_ids_m][:,1:1863]
Y = Y[tr_ids_m][:,1:1863]

X4 = np.expand_dims(X4,-1)
X4.shape

tmp = np.load('./Data/Disordered_Regions_Test_Dataset.npz')
test_ids = []
with open('./Data/SL329_id.txt') as f:
    for l in f.readlines():
        k = l.split('|')[1].split()[0]
        test_ids += [k]
te_ids_m = [True if i in test_ids else False for i in tmp['seq_ids']]
X1_te = np.argmax(tmp['sequences'],axis=-1)[te_ids_m][:,1:1863]
X2_te = tmp['pssms'][te_ids_m][:,1:1863]
Y_te = tmp['regions'][te_ids_m][:,1:1863]
X_m_te = X1_te!=0
X4_te = X_m_te.astype(np.int32)
X4_te = np.expand_dims(X4_te,-1)
X4_te.shape

(318, 1862, 1)

In [6]:
X1_temp = []
for seq in X1:
    end = np.where(seq==23)[0]
    if len(end)==0:
        end = len(seq)
    else:
        end = end[0]
    X1_temp.append([embedding_keys[num_aminoAcids[seq[idx]-1]] for idx in range(0,end)])
    X1_temp[-1].extend([0]*(2000-len(X1_temp[-1])))
    X1_temp[-1] = np.array(X1_temp[-1])
X1_temp = np.array(X1_temp)

X1_val_temp = []
for seq in X1_val:
    end = np.where(seq==23)[0]
    if len(end)==0:
        end = len(seq)
    else:
        end = end[0]
    X1_val_temp.append([embedding_keys[num_aminoAcids[seq[idx]-1]] for idx in range(0,end)])
    X1_val_temp[-1].extend([0]*(2000-len(X1_val_temp[-1])))
    X1_val_temp[-1] = np.array(X1_val_temp[-1])
X1_val_temp = np.array(X1_val_temp)

X1_te_temp = []
for seq in X1_te:
    end = np.where(seq==23)[0]
    if len(end)==0:
        end = len(seq)
    else:
        end = end[0]
    X1_te_temp.append([embedding_keys[num_aminoAcids[seq[idx]-1]] for idx in range(0,end)])
    X1_te_temp[-1].extend([0]*(2000-len(X1_te_temp[-1])))
    X1_te_temp[-1] = np.array(X1_te_temp[-1])
X1_te_temp = np.array(X1_te_temp)

X1_temp_oneHot = np.array(tf.one_hot(X1_temp,len(embedding_keys))[:,:,1:])
X1_val_temp_oneHot = np.array(tf.one_hot(X1_val_temp,len(embedding_keys))[:,:,1:])
X1_te_temp_oneHot = np.array(tf.one_hot(X1_te_temp,len(embedding_keys))[:,:,1:])

2023-04-30 12:46:00.607301: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-30 12:46:00.630441: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-30 12:46:00.631002: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-30 12:46:00.631860: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

In [7]:
class embeddingLoader(tfk.utils.Sequence):
    def __init__(self,path,pssms, masks,Y_oneHot,batch_size=30,shuffle=True):
        self.path = path
        self.pssms = pssms
        self.masks = masks
        self.Y_oneHot = copy.deepcopy(np.array(Y_oneHot))
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.length = len(Y_oneHot)
        self.indices = np.arange(0,self.length)
        self.on_epoch_end()

    def generate_batches(self):
        self.Y_oneHot_ = self.Y_oneHot[self.indices]
        self.pssms_ = self.pssms[self.indices]
        self.masks_ = self.masks[self.indices]
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)
        self.generate_batches()

    def __getitem__(self,index):
        token_representations = []
        for idx in self.indices[index*self.batch_size:(index+1)*self.batch_size]:
            token_representations.append(np.load(self.path+str(idx).zfill(6)+'.npy')[0])
        return ([
                tf.convert_to_tensor(np.array(token_representations)),
                self.masks_[index*self.batch_size:(index+1)*self.batch_size],
                self.pssms_[index*self.batch_size:(index+1)*self.batch_size]
                ],
                self.Y_oneHot_[index*self.batch_size:(index+1)*self.batch_size]
                )

    def __len__(self):

        if self.length%self.batch_size!=0:
            return 1+(self.length//self.batch_size)
        return self.length//self.batch_size

In [8]:
padded_Y = np.zeros((3000, 2000, 2))
padded_Y[:Y.shape[0],:Y.shape[1],:Y.shape[2]] = Y

padded_Y_val = np.zeros((1229, 2000, 2))
padded_Y_val[:Y_val.shape[0],:Y_val.shape[1],:Y_val.shape[2]] = Y_val


padded_Y_te = np.zeros((318, 2000, 2))
padded_Y_te[:Y_te.shape[0],:Y_te.shape[1],:Y_te.shape[2]] = Y_te

padded_X2 = np.zeros((3000, 2000, 20))
padded_X2[:X2.shape[0],:X2.shape[1],:X2.shape[2]] = X2

padded_X2_val = np.zeros((1229, 2000, 20))
padded_X2_val[:X2_val.shape[0],:X2_val.shape[1],:X2_val.shape[2]] = X2_val


padded_X2_te = np.zeros((318, 2000, 20))
padded_X2_te[:X2_te.shape[0],:X2_te.shape[1],:X2_te.shape[2]] = X2_te

padded_X4 = np.zeros((3000, 2000, 1))
padded_X4[:X4.shape[0],:X4.shape[1],:X4.shape[2]] = X4

padded_X4_val = np.zeros((1229, 2000, 1))
padded_X4_val[:X4_val.shape[0],:X4_val.shape[1],:X4_val.shape[2]] = X4_val


padded_X4_te = np.zeros((318, 2000, 1))
padded_X4_te[:X4_te.shape[0],:X4_te.shape[1],:X4_te.shape[2]] = X4_te

In [9]:
with strategy.scope():
    input1_ = tfk.layers.Input(shape=(2000,300, ), name='sequence_input')
    input2_ = tfk.layers.Input(shape=(2000,1, ), name='mask')
    input3_ = tfk.layers.Input(shape=(2000,20, ), name='pssm')

    x = tfk.layers.Multiply()([input1_, input2_])
    x = tfk.layers.concatenate([x, input3_], axis=-1)
    x = tfk.layers.Bidirectional(tfk.layers.GRU(units=150, return_sequences=True), name='bigru1')(x)
    x = tfk.layers.Multiply()([x, input2_])
    x = tfk.layers.Bidirectional(tfk.layers.GRU(units=150, return_sequences=True), name='bigru2')(x)
    x = tfk.layers.Multiply()([x, input2_])
    
    x = tfk.layers.TimeDistributed(tfk.layers.Dense(500, activation='relu'), name='output1')(x)
    x = tfk.layers.Multiply()([x, input2_])
    x = tfk.layers.TimeDistributed(tfk.layers.Dense(500, activation='relu'), name='output2')(x)
    output_ = tfk.layers.TimeDistributed(tfk.layers.Dense(2, activation='softmax') ,name='output')(x)
    
    conv_model = tfk.models.Model([input1_, input2_, input3_], output_)
    conv_model.compile(loss=NonZeroCCE(), metrics=[Accuracy], optimizer='adam', run_eagerly = True)
    conv_model.summary()
conv_model.load_weights("./Weights/conv_pssm_model.h5")

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 sequence_input (InputLayer)    [(None, 2000, 300)]  0           []                               
                                                                                                  
 mask (InputLayer)              [(None, 2000, 1)]    0           []                               
                                                                                                  
 multiply (Multiply)            (None, 2000, 300)    0           ['sequence_input[0][0]',         
                                                                  'mask[0][0]']                   
                                                                                                  
 pssm (InputLayer)              [(None, 2000, 20)]   0           []                           

In [10]:
print("*******************************************Conv ONLY*******************************************")
print()
print()
val_loader = embeddingLoader("./Data/Embeddings/Conv/val/",padded_X2_val,padded_X4_val,padded_Y_val,30,False)
print("*******************************************VALIDATION*******************************************")
preds = conv_model.predict(val_loader, verbose=1)

m = np.sum(padded_Y_val, axis=-1)
y_t = np.argmax(padded_Y_val[m==1],axis=-1).flatten()
y_p = np.argmax(preds[m==1],axis=-1).flatten()
print(classification_report(y_t,y_p))
t = confusion_matrix(y_t,y_p)
print(t,accuracy_score(y_t,y_p))
tp = np.array(t[1][1],dtype=np.float64)
fp = np.array(t[0][1],dtype=np.float64)
tn = np.array(t[0][0],dtype=np.float64)
fn = np.array(t[1][0],dtype=np.float64)
sp = tp/(tp+fn)
sn = tn/(tn+fp)
bacc = (sp+sn)/2
mcc = ((tp*tn)-(fp*fn))/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
print(sp,sn,bacc,mcc,f1_score(y_t,y_p,average='weighted'))

test_loader = embeddingLoader("./Data/Embeddings/Conv/test/",padded_X2_te, padded_X4_te,padded_Y_te,30,False)
print()
print()
print("*******************************************Test*******************************************")
preds = conv_model.predict(test_loader, verbose=1)

m = np.sum(padded_Y_te, axis=-1)
y_t = np.argmax(padded_Y_te[m==1],axis=-1).flatten()
y_p = np.argmax(preds[m==1],axis=-1).flatten()
print(classification_report(y_t,y_p))
t = confusion_matrix(y_t,y_p)
print(t,accuracy_score(y_t,y_p))
tp = np.array(t[1][1],dtype=np.float64)
fp = np.array(t[0][1],dtype=np.float64)
tn = np.array(t[0][0],dtype=np.float64)
fn = np.array(t[1][0],dtype=np.float64)
sp = tp/(tp+fn)
sn = tn/(tn+fp)
bacc = (sp+sn)/2
mcc = ((tp*tn)-(fp*fn))/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
print(sp,sn,bacc,mcc,f1_score(y_t,y_p,average='weighted'))


*******************************************Conv ONLY*******************************************


*******************************************VALIDATION*******************************************


2023-04-30 12:48:23.248028: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401


41/41 [==============================] - 42s 1s/step
              precision    recall  f1-score   support

           0       0.94      0.99      0.96    276748
           1       0.81      0.41      0.55     29082

    accuracy                           0.93    305830
   macro avg       0.87      0.70      0.76    305830
weighted avg       0.93      0.93      0.93    305830

[[273837   2911]
 [ 17031  12051]] 0.9347938397148743
0.4143800288838457 0.9894814054663449 0.7019307171750953 0.549216993433059 0.9251527004027899


*******************************************Test*******************************************
11/11 [==============================] - 10s 980ms/step
              precision    recall  f1-score   support

           0       0.78      0.96      0.86     47388
           1       0.91      0.63      0.75     34221

    accuracy                           0.82     81609
   macro avg       0.85      0.79      0.80     81609
weighted avg       0.84      0.82      0.81     816

In [11]:
with strategy.scope():
    input1_ = tfk.layers.Input(shape=(2000,300, ), name='sequence_input')
    input2_ = tfk.layers.Input(shape=(2000,1, ), name='mask')
    input3_ = tfk.layers.Input(shape=(2000,20, ), name='pssm')
    
    x = tfk.layers.Multiply()([input1_, input2_])
    x = tfk.layers.concatenate([x, input3_], axis=-1)
    x = tfk.layers.Bidirectional(tfk.layers.GRU(units=150, return_sequences=True), name='bigru1')(x)
    x = tfk.layers.Multiply()([x, input2_])
    x = tfk.layers.Bidirectional(tfk.layers.GRU(units=150, return_sequences=True), name='bigru2')(x)
    x = tfk.layers.Multiply()([x, input2_])    
    x = tfk.layers.TimeDistributed(tfk.layers.Dense(500, activation='relu'), name='output1')(x)
    x = tfk.layers.Multiply()([x, input2_])
    x = tfk.layers.TimeDistributed(tfk.layers.Dense(500, activation='relu'), name='output2')(x)
    output_ = tfk.layers.TimeDistributed(tfk.layers.Dense(2, activation='softmax') ,name='output')(x)
    
    gru_model = tfk.models.Model([input1_, input2_, input3_], output_)
    gru_model.compile(loss=NonZeroCCE(), metrics=[Accuracy], optimizer='adam', run_eagerly = True)
    gru_model.summary()
gru_model.load_weights("./Weights/gru_pssm_model.h5")

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 sequence_input (InputLayer)    [(None, 2000, 300)]  0           []                               
                                                                                                  
 mask (InputLayer)              [(None, 2000, 1)]    0           []                               
                                                                                                  
 multiply_4 (Multiply)          (None, 2000, 300)    0           ['sequence_input[0][0]',         
                                                                  'mask[0][0]']                   
                                                                                                  
 pssm (InputLayer)              [(None, 2000, 20)]   0           []                         

In [12]:
print("*******************************************GRU ONLY*******************************************")
print()
print()
val_loader = embeddingLoader("./Data/Embeddings/GRU/val/",padded_X2_val,padded_X4_val,padded_Y_val,30,False)
print("*******************************************VALIDATION*******************************************")
preds = gru_model.predict(val_loader, verbose=1)

m = np.sum(padded_Y_val, axis=-1)
y_t = np.argmax(padded_Y_val[m==1],axis=-1).flatten()
y_p = np.argmax(preds[m==1],axis=-1).flatten()
print(classification_report(y_t,y_p))
t = confusion_matrix(y_t,y_p)
print(t,accuracy_score(y_t,y_p))
tp = np.array(t[1][1],dtype=np.float64)
fp = np.array(t[0][1],dtype=np.float64)
tn = np.array(t[0][0],dtype=np.float64)
fn = np.array(t[1][0],dtype=np.float64)
sp = tp/(tp+fn)
sn = tn/(tn+fp)
bacc = (sp+sn)/2
mcc = ((tp*tn)-(fp*fn))/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
print(sp,sn,bacc,mcc,f1_score(y_t,y_p,average='weighted'))

test_loader = embeddingLoader("./Data/Embeddings/GRU/test/",padded_X2_te, padded_X4_te,padded_Y_te,30,False)
print()
print()
print("*******************************************Test*******************************************")
preds = gru_model.predict(test_loader, verbose=1)

m = np.sum(padded_Y_te, axis=-1)
y_t = np.argmax(padded_Y_te[m==1],axis=-1).flatten()
y_p = np.argmax(preds[m==1],axis=-1).flatten()
print(classification_report(y_t,y_p))
t = confusion_matrix(y_t,y_p)
print(t,accuracy_score(y_t,y_p))
tp = np.array(t[1][1],dtype=np.float64)
fp = np.array(t[0][1],dtype=np.float64)
tn = np.array(t[0][0],dtype=np.float64)
fn = np.array(t[1][0],dtype=np.float64)
sp = tp/(tp+fn)
sn = tn/(tn+fp)
bacc = (sp+sn)/2
mcc = ((tp*tn)-(fp*fn))/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
print(sp,sn,bacc,mcc,f1_score(y_t,y_p,average='weighted'))


*******************************************GRU ONLY*******************************************


*******************************************VALIDATION*******************************************
41/41 [==============================] - 43s 1s/step
              precision    recall  f1-score   support

           0       0.95      0.98      0.96    276748
           1       0.74      0.50      0.59     29082

    accuracy                           0.94    305830
   macro avg       0.84      0.74      0.78    305830
weighted avg       0.93      0.94      0.93    305830

[[271583   5165]
 [ 14616  14466]] 0.9353202759703103
0.49742108520734474 0.9813368118288117 0.7393789485180782 0.5730137767904264 0.9295888283469718


*******************************************Test*******************************************
11/11 [==============================] - 10s 1s/step
              precision    recall  f1-score   support

           0       0.82      0.93      0.87     47388
           1       0.

In [13]:
with strategy.scope():
    input1_ = tfk.layers.Input(shape=(2000,600, ), name='sequence_input')
    input2_ = tfk.layers.Input(shape=(2000,1, ), name='mask')
    input3_ = tfk.layers.Input(shape=(2000,20, ), name='pssm')
    
    x = tfk.layers.Multiply()([input1_, input2_])
    x = tfk.layers.concatenate([x, input3_], axis=-1)
    x = tfk.layers.Bidirectional(tfk.layers.GRU(units=150, return_sequences=True), name='bigru1')(x)
    x = tfk.layers.Multiply()([x, input2_])
    x = tfk.layers.Bidirectional(tfk.layers.GRU(units=150, return_sequences=True), name='bigru2')(x)
    x = tfk.layers.Multiply()([x, input2_])    
    x = tfk.layers.TimeDistributed(tfk.layers.Dense(500, activation='relu'), name='output1')(x)
    x = tfk.layers.Multiply()([x, input2_])
    x = tfk.layers.TimeDistributed(tfk.layers.Dense(500, activation='relu'), name='output2')(x)
    output_ = tfk.layers.TimeDistributed(tfk.layers.Dense(2, activation='softmax') ,name='output')(x)
    
    conv_GRU_model = tfk.models.Model([input1_, input2_, input3_], output_)
    conv_GRU_model.compile(loss=NonZeroCCE(), metrics=[Accuracy], optimizer='adam', run_eagerly = True)
    conv_GRU_model.summary()
conv_GRU_model.load_weights("./Weights/conv_gru_pssm_model.h5")

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 sequence_input (InputLayer)    [(None, 2000, 600)]  0           []                               
                                                                                                  
 mask (InputLayer)              [(None, 2000, 1)]    0           []                               
                                                                                                  
 multiply_8 (Multiply)          (None, 2000, 600)    0           ['sequence_input[0][0]',         
                                                                  'mask[0][0]']                   
                                                                                                  
 pssm (InputLayer)              [(None, 2000, 20)]   0           []                         

In [14]:
print("*******************************************Conv_GRU ONLY*******************************************")
print()
print()
val_loader = embeddingLoader("./Data/Embeddings/Conv_GRU/val/",padded_X2_val,padded_X4_val,padded_Y_val,30,False)
print("*******************************************VALIDATION*******************************************")
preds = conv_GRU_model.predict(val_loader, verbose=1)

m = np.sum(padded_Y_val, axis=-1)
y_t = np.argmax(padded_Y_val[m==1],axis=-1).flatten()
y_p = np.argmax(preds[m==1],axis=-1).flatten()
print(classification_report(y_t,y_p))
t = confusion_matrix(y_t,y_p)
print(t,accuracy_score(y_t,y_p))
tp = np.array(t[1][1],dtype=np.float64)
fp = np.array(t[0][1],dtype=np.float64)
tn = np.array(t[0][0],dtype=np.float64)
fn = np.array(t[1][0],dtype=np.float64)
sp = tp/(tp+fn)
sn = tn/(tn+fp)
bacc = (sp+sn)/2
mcc = ((tp*tn)-(fp*fn))/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
print(sp,sn,bacc,mcc,f1_score(y_t,y_p,average='weighted'))

test_loader = embeddingLoader("./Data/Embeddings/Conv_GRU/test/",padded_X2_te, padded_X4_te,padded_Y_te,30,False)
print()
print()
print("*******************************************Test*******************************************")
preds = conv_GRU_model.predict(test_loader, verbose=1)

m = np.sum(padded_Y_te, axis=-1)
y_t = np.argmax(padded_Y_te[m==1],axis=-1).flatten()
y_p = np.argmax(preds[m==1],axis=-1).flatten()
print(classification_report(y_t,y_p))
t = confusion_matrix(y_t,y_p)
print(t,accuracy_score(y_t,y_p))
tp = np.array(t[1][1],dtype=np.float64)
fp = np.array(t[0][1],dtype=np.float64)
tn = np.array(t[0][0],dtype=np.float64)
fn = np.array(t[1][0],dtype=np.float64)
sp = tp/(tp+fn)
sn = tn/(tn+fp)
bacc = (sp+sn)/2
mcc = ((tp*tn)-(fp*fn))/np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
print(sp,sn,bacc,mcc,f1_score(y_t,y_p,average='weighted'))


*******************************************Conv_GRU ONLY*******************************************


*******************************************VALIDATION*******************************************
41/41 [==============================] - 83s 2s/step
              precision    recall  f1-score   support

           0       0.94      0.99      0.96    276748
           1       0.80      0.42      0.55     29082

    accuracy                           0.93    305830
   macro avg       0.87      0.71      0.76    305830
weighted avg       0.93      0.93      0.93    305830

[[273667   3081]
 [ 16804  12278]] 0.9349802177680411
0.4221855443229489 0.9888671282177288 0.7055263362703389 0.552101588424858 0.9257279692704664


*******************************************Test*******************************************
11/11 [==============================] - 21s 2s/step
              precision    recall  f1-score   support

           0       0.80      0.95      0.87     47388
           1      